In [1]:
import warnings
warnings.filterwarnings('ignore')

#import psycopg2
import csv
import re
import time
from fuzzywuzzy import fuzz
from fuzzywuzzy import process 
import math
from collections import Counter
import datetime
import pandas as pd
import numpy as np
from math import acos, cos, asin, sin, atan2, tan, radians

In [2]:
cleanup_lithology_file='./thesaurus_cleanup.csv'
litho_dic_file='./litho_dico_test.csv'

lithology_file='./test.csv'

## Text Pre-processing Functions

In [3]:
cleanup_dic_list=[]
def clean_up():
    cur = pd.read_csv(cleanup_lithology_file,encoding = "ISO-8859-1", dtype='object')
    cur=cur.values.tolist() 
    for record in cur:
        cleanup_dic_list.append(record)
def clean_text(text):
    text=text.lower()
    text=(re.sub('\(.*\)', '', text)) # removes text in parentheses
    text=(re.sub('\[.*\]', '', text)) # removes text in parentheses
    text=(re.sub('\{.*\}', '', text)) # removes text in parentheses
    text=text.replace('>',' ').replace('?',' ').replace('/',' ') # removes >, ?, and /
    text = text.replace('<',' ').replace('\\',' ')
    text = text.replace('.',' ').replace(',',' ').replace(';',' ').replace(':',' ')
    text = text.replace('%',' ').replace('-',' ').replace('_',' ')
    text = text.replace('$', ' ').replace('@',' ').replace('~',' ').replace('^',' ')
    text = text.replace('!',' ').replace('#',' ').replace('*',' ').replace('=',' ')
    text = text.replace('"',' ').replace('|',' ').replace('*',' ').replace('=',' ')
    text = text.replace('\n',' ').replace('\r',' ').replace('\t',' ')
    #if text.isnumeric(): # if the text is number
    #    text = re.sub('\d', ' ', text) #replace numbers
    text=(re.sub(r'[0-9]+', '', text)) # removes numbers
    text = text.split('with', 1)[0]  #参数[n]表示选择结果列表中的第n个(0是第一个)片
    text = text.split('w/', 1)[0]
    text = text.split('/w', 1)[0]
    text = text.split(' in ', 1)[0] #vein #ferruginous #actinolite #alkaline #cainzoic #fine #dominant
    #text = text.split(' and ', 1)[0] #banded #andesite #sand #sandstone
    text = text.split('within', 1)[0]
    text = text.split('+', 1)[0]
    text = text.split('&', 1)[0]
    text = text.split('adjacent to', 1)[0]
    text = text.split('poss', 1)[0]
    text = text.split('possibly', 1)[0]
    text = text.split('prob', 1)[0]
    text = text.split('probable', 1)[0]
    text = text.split('probably', 1)[0]
    text = text.split('back to', 1)[0]
    text = text.split('due to', 1)[0]
    text = text.split('look', 1)[0]
    text = text.split('looks', 1)[0]
    text = text.split('maybe', 1)[0]
    text = text.split('same', 1)[0]
    text = text.split('similar', 1)[0]
    text = text.split('slowly changing', 1)[0]
    text = text.split('changed', 1)[0]
    text = text.split('slowly', 1)[0]
    text = text.split('like', 1)[0]

    for cleanup_dic_ele in cleanup_dic_list:
        cleaned_item =str(cleanup_dic_ele).replace('(','').replace(')','').replace(',','').replace('\'','')
        #print (cleaned_item)
        #cleaned_item =str(cleanup_dic_ele)
        text = text.replace(cleaned_item,'')
    return text

def tokenize(text, min_len=1):
    '''Function that tokenize a set of strings
    Input:
        -text: set of strings
        -min_len: tokens length
    Output:
        -list containing set of tokens'''

    tokens = [word.lower() for sent in nltk.sent_tokenize(text)
              for word in nltk.word_tokenize(sent)]
    filtered_tokens = []

    for token in tokens:
        if token.isalpha() and len(token) >= min_len:
            filtered_tokens.append(token)

    return [x.lower() for x in filtered_tokens if x not in stop]


def tokenize_and_lemma(text, min_len=0):
    '''Function that retrieves lemmatised tokens
    Inputs:
        -text: set of strings
        -min_len: length of text
    Outputs:
        -list containing lemmatised tokens'''
    filtered_tokens = tokenize(text, min_len=min_len)

    lemmas = [lemma.lemmatize(t) for t in filtered_tokens]
    return lemmas

litho_dico=[]
def litho_dic():
    cur=pd.read_csv(litho_dic_file,encoding = "ISO-8859-1", dtype='object')
    cur=cur.values.tolist() 
    for record in cur:
        litho_dico.append(record)

In [4]:
#attributecolumn, attributevalue, 
#colllarid, from, to, description

In [5]:
Comments_dic=[]
def Comments_Dic():
    cur=pd.read_csv(lithology_file,encoding = "ISO-8859-1", dtype='object')
    cur=cur.values.tolist() 
    for record in cur:
        Comments_dic.append(record)


In [6]:
Comments_fuzzy=[]
def Comments_With_fuzzy():
    bestmatch=-1
    bestlitho=''
    top=[]
    i=0
    comments_sub_list=[]
    fieldnames=['Collar ID', 'FROM', 'TO', 'DESCRIPTION','DESCRIPTION_CLEAN','LITHOLOGY','FUZZY_SCORE']
    out= open("lithology_fuzzy.csv", "w",encoding ="utf-8")
    for ele in fieldnames:
        out.write('%s,' %ele)
    out.write('\n')
    Comments_Dic_new = [list(elem) for elem in Comments_dic]
    for Comments_Dic_ele in Comments_Dic_new:
        cleaned_text=clean_text(Comments_Dic_ele[3])
        
        words=(re.sub('\(.*\)', '', cleaned_text)).strip() 
        words=words.rstrip('\n\r').split(" ")
        last=len(words)-1 #position of last word in phrase
        
        for litho_dico_ele in litho_dico:
            litho_words=str(litho_dico_ele).lower().rstrip('\n\r').replace('(','').replace(')','').replace('\'','').replace(',','').split(" ")

            scores=process.extract(cleaned_text, litho_words, scorer=fuzz.token_set_ratio)#fuzz.token_set_ratio/fuzz.partial_ratio/  # Matching
            for sc in scores:                        
                if(sc[1]>bestmatch): #better than previous best match
                    bestmatch =  sc[1]
                    bestlitho=litho_words[0]
                    top.append([sc[0],sc[1]])
                    if(sc[0]==words[last]): #bonus for being last word in phrase
                        bestmatch=bestmatch*1.01
                elif (sc[1]==bestmatch): #equal to previous best match
                    if(sc[0]==words[last]): #bonus for being last word in phrase
                        bestlitho=litho_words[0]
                        bestmatch=bestmatch*1.01
                    else:
                        top.append([sc[0],sc[1]])

        i=0
        if bestmatch >80:
            #Comments_fuzzy.append([Comments_Dic_ele[0],Comments_Dic_ele[1],cleaned_text,bestlitho,bestmatch]) #top_new[1]])  or top[0][1]
            out.write('%s,' %Comments_Dic_ele[0].replace('(','').replace(')','').replace('\'','').replace(',','').replace(',' , '').replace('\n',''))
            out.write('%s,' %Comments_Dic_ele[1])
            out.write('%s,' %Comments_Dic_ele[2])
            out.write('%s,' %Comments_Dic_ele[3].replace('(','').replace(')','').replace('\'','').replace(',','').replace('\n',''))
            out.write('%s,' %cleaned_text)   #.replace('(','').replace(')','').replace('\'','').replace(',','').replace('\n',''))
            out.write('%s,' %bestlitho.replace('(','').replace(')','').replace('[','').replace(']','').replace('\'','').replace(',','').replace('\n',''))
            out.write('%d,' %bestmatch)
            out.write('\n')
            top.clear()
            CET_Litho=''
            bestmatch=-1
            bestlitho=''
        else:
            #Comments_fuzzy.append([Comments_Dic_ele[0],Comments_Dic_ele[1],cleaned_text,'Other',bestmatch])  #top_new[1]])
            out.write('%s,' %Comments_Dic_ele[0].replace('(','').replace(')','').replace('\'','').replace(',','').replace(',' , '').replace('\n',''))
            out.write('%s,' %Comments_Dic_ele[1])
            out.write('%s,' %Comments_Dic_ele[2])
            out.write('%s,' %Comments_Dic_ele[3].replace('(','').replace(')','').replace('\'','').replace(',','').replace('\n',''))
            out.write('%s,' %cleaned_text)   #.replace('(','').replace(')','').replace('\'','').replace(',','').replace('\n',''))
            out.write('Other,')
            out.write('%d,' %bestmatch)
            out.write('\n')
            top.clear()
            CET_Litho=''
            bestmatch=-1
            bestlitho=''

In [7]:
litho_dic()
clean_up()
Comments_Dic()
Comments_With_fuzzy()